In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.functions import split
import pandas as pd
import matplotlib.pyplot as plt
import upsetplot


#créer une session dans le master
#spark = SparkSession.builder.master("spark://172.20.53.96:7077").appName("WDC-structuredness").getOrCreate()
spark = SparkSession.builder.master("local").appName("WDC-structuredness").getOrCreate()
        
#fichiers de config qui permettent de se connecter au serveur de stockage s3 qui contient les fichiers de DataCommons
endpoint_url = 'https://s3.os-bird.glicid.fr/'
aws_access_key_id = '***REMOVED***'
aws_secret_access_key = '***REMOVED***'
hadoopConf = spark._jsc.hadoopConfiguration()
hadoopConf.set('fs.s3a.access.key', aws_access_key_id)
hadoopConf.set('fs.s3a.secret.key', aws_secret_access_key)
hadoopConf.set('fs.s3a.endpoint', endpoint_url)
hadoopConf.set('fs.s3a.path.style.access', 'true')
hadoopConf.set('fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')

hadoopConf.set('spark.worker.cleanup.enabled', 'true')
hadoopConf.set('fs.s3a.committer.name', 'magic')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/07 13:55:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/07 13:55:14 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
23/04/07 13:55:14 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
readall = spark.read.option("header",True) \
  .csv("s3a://test-out/wdcfix/**")

readall.take(5)

import pyspark.sql.functions as f
csall=readall.groupby("pset").agg(f.sum("count").alias('count')).sort(f.desc("count"))

csall.createOrReplaceTempView("CSET")
csall.show(50,truncate=100)

23/04/07 13:55:15 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


+----------------------------------------------------------------------------------------------------+------------+
|                                                                                                pset|       count|
+----------------------------------------------------------------------------------------------------+------------+
|                                   <schema.org/item>|<schema.org/position>|isa:<schema.org/ListItem>|6.82470722E8|
|                                        <schema.org/itemListElement>|isa:<schema.org/BreadcrumbList>|4.56613881E8|
|                 <schema.org/item>|<schema.org/name>|<schema.org/position>|isa:<schema.org/ListItem>|3.93001409E8|
|                          <schema.org/query-input>|<schema.org/target>|isa:<schema.org/SearchAction>|3.71206005E8|
|                                                     <schema.org/target>|isa:<schema.org/ReadAction>|2.43969439E8|
|                <schema.org/height>|<schema.org/url>|<schema.org/width>

In [ ]:
def structuredness(cname):
    sets=spark.sql("select pset,count from CSET where pset like '%isa:<schema.org/"+cname+">%' ")
    sets = sets.withColumn("pset", split(sets["pset"], "\\|"))
    sets.createOrReplaceTempView("sets")
    top_sets= spark.sql("""
    SELECT  *
    FROM sets
    ORDER BY count DESC
    LIMIT 20
    """)
    #top_sets.show(truncate=0)
    
    exploded_data = top_sets.select("pset", "count") \
                    .selectExpr("pset", "count", "explode(pset) as predicate")
    distinct_predicate_count = exploded_data.select("predicate") \
                                       .distinct() \
                                       .count()
    print(distinct_predicate_count)
    
    querry = spark.sql("SELECT SUM(count) as count_sum FROM sets")
    totalcount = querry.first()["count_sum"]
    print(totalcount)
    
    totalpredcount = totalcount*distinct_predicate_count
    print(totalpredcount)
    
    querry = spark.sql("SELECT SUM(count * size(pset)) as count_sum  FROM sets")
    countused = querry.first()["count_sum"]
    print(countused)
    
    print("struturedness:")
    structuredness = countused/totalpredcount
    print(structuredness)

In [ ]:
structuredness("BioChemEntity")